# Substring Kernel

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

### Import Data

In [2]:
def train_val_raw(i):
    tr1 = np.array(pd.read_csv('data/Xtr{}.csv'.format(i))["seq"])
    tr1_labels = np.array(pd.read_csv('data/Ytr{}.csv'.format(i))["Bound"])
    
    VALIDATION_RATIO = 0.2

    shuffled_idx = np.arange(tr1.shape[0])
    np.random.shuffle(shuffled_idx)
    
    raw_val, raw_train = tr1[shuffled_idx[:int(VALIDATION_RATIO*tr1.shape[0])]], tr1[shuffled_idx[int(VALIDATION_RATIO*tr1.shape[0]):]]
    labels_val, labels_train = tr1_labels[shuffled_idx[:int(VALIDATION_RATIO*tr1.shape[0])]], tr1_labels[shuffled_idx[int(VALIDATION_RATIO*tr1.shape[0]):]]
    
    return raw_train, raw_val, labels_train, labels_val

In [3]:
x1_train, x1_val, x1_train_labels, x1_val_labels = train_val_raw(0)

### Original Implementation

In [6]:
def computeKernel(s1, s2, n, lambd = 0.9):
    n1 = len(s1)
    n2 = len(s2)
    
    prev = np.zeros((n1, n2))
    nxt = np.zeros((n1, n2))
    K = np.zeros(n)
    for i in range(n1):
        for j in range(n2):
            if s1[i] == s2[j]:
                prev[i,j] = lambd**2
                K[0] += prev[i,j]

    K[0] /= lambd ** 2
    for l in range(1, n):
        B = np.zeros((n1, n2))

        for i in range(0, n1):
            for j in range(0,n2):
                B[i,j] = prev[i,j]
                if i != 0: # Problem w/ -1 indexing in Python....
                    B[i,j] += lambd * B[i-1,j]
                if j!=0:
                    B[i,j] += lambd * B[i,j-1]
                if i!=0 and j!=0:
                    B[i,j] -= lambd**2 * B[i-1, j-1]
                    
                if s1[i] == s2[j] and i-1 !=0 and j-1 != 0 :
                    nxt[i,j] = lambd ** 2 * B[i-1,j-1]
                    K[l] += nxt[i,j]

        K[l] /= lambd ** (2*(l+1))
        prev = nxt.copy()
    return K[-1]

In [ ]:
sub_K = np.zeros((x1_train.shape[0], x1_train.shape[0]))
for i in tqdm(range(x1_train.shape[0])):
    for j in range(i, x1_train.shape[0]):
        sub_K[i,j] = computeKernel(x1_train[i],x1_train[j],3)

  0%|          | 1/1600 [01:09<30:51:19, 69.47s/it]

### Vectorized Implementation

In [4]:
def computeKernelVect(Comps, n, lambd = 0.9):
    # Initialize
    length = Comps.shape[2]
    K = np.zeros((Comps.shape[0], Comps.shape[1], n))
    K[:,:,0] += np.sum(Comps, axis = (2,3))
    prev = lambd**2 * Comps.copy()
    
    # Iterate over all letters
    for l in range(1, n):
        B = np.zeros((Comps.shape[0], Comps.shape[1], length, length))
        nxt = np.zeros((Comps.shape[0], Comps.shape[1], length, length))
        for i in range(0, length):
            for j in range(0,length):
                B[:,:,i,j] = prev[:,:,i,j]
                if i != 0: # Problem w/ -1 indexing in Python....
                    B[:,:,i,j] += lambd * B[:,:,i-1,j]
                if j!=0:
                    B[:,:,i,j] += lambd * B[:,:,i,j-1]
                if i!=0 and j!=0:
                    B[:,:,i,j] -= lambd**2 * B[:,:,i-1, j-1]
                if i-1 !=0 and j-1 != 0 :
                    nxt[:,:,i,j] = Comps[:,:,i,j] * lambd ** 2 * B[:,:,i-1,j-1]
                    K[:,:,l] += nxt[:,:,i,j]

        K[:,:,l] /= lambd ** (2*(l+1))
        prev = nxt.copy()
    return K[:,:,-1]

In [6]:
# Initialize
C = np.array([list(s) for s in x1_train])
slicing = np.linspace(0, C.shape[0], 400).astype(int)

# Compute K for each slice
sub_K = np.zeros((x1_train.shape[0], x1_train.shape[0]))
for i in tqdm(range(len(slicing) - 1), position = 0, leave = True):
    Comps = (C[slicing[i]:slicing[i+1], None, :, None] == C[None,:,None,:])
    sub_K[slicing[i]:slicing[i+1],:] = computeKernelVect(Comps,3)

  0%|          | 1/399 [01:26<9:30:33, 86.01s/it]

KeyboardInterrupt: 